# ETL Silver → Gold
Popula o Star Schema no PostgreSQL a partir dos dados limpos do Silver Layer.

## 1. Imports e configurações

In [1]:
import psycopg2
from psycopg2.extras import execute_batch
import pandas as pd

DB_CONFIG = {
    "host": "localhost",
    "port": 5432,
    "database": "fake-real-job",
    "user": "postgres",
    "password": "senha123",
}

conn = psycopg2.connect(**DB_CONFIG)
cur = conn.cursor()

print("=" * 60)
print("DIAGNOSTICO PRE-ETL (SILVER_JOBS -> GOLD)")
print("=" * 60)

cur.execute("SELECT COUNT(*) FROM public.silver_jobs;")
total = cur.fetchone()[0]

cur.execute("SELECT COUNT(*) FROM public.silver_jobs WHERE salary_avg IS NOT NULL;")
com_salario = cur.fetchone()[0]

cur.execute("SELECT COUNT(*) FROM public.silver_jobs WHERE is_fake IS TRUE;")
fake = cur.fetchone()[0]

print(f"Total vagas: {total:,}")
print(f"Com salario: {com_salario:,}")
print(f"Vagas fake: {fake:,}")

print("=" * 60)


DIAGNOSTICO PRE-ETL (SILVER_JOBS -> GOLD)
Total vagas: 3,000
Com salario: 2,197
Vagas fake: 1,472


In [2]:
df = pd.read_sql("SELECT * FROM public.silver_jobs", conn)
df.head()


/var/folders/_s/0v_8sh5d24vf52_79rzxfcl40000gn/T/ipykernel_4554/861218001.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM public.silver_jobs", conn)


,job_id,job_title,company_name,required_experience_years,is_fake,fraud_reason,posting_timestamp,application_deadline_timestamp,salary_min,salary_max,salary_avg,remote,country,state
0,1,Software Engineer,Company_543,8,False,None,2023-11-24,2024-09-16,60000.0,80000.0,70000.0,False,Toronto,Canada
1,2,Content Writer,Company_192,10,False,None,2023-03-03,2024-10-18,40000.0,60000.0,50000.0,False,Toronto,Canada
2,3,Customer Support Specialist,None,3,True,Suspicious email,2023-07-31,2024-01-13,60000.0,80000.0,70000.0,True,None,None
3,4,Data Analyst,Company_95,10,False,None,2023-08-14,2024-02-09,NaN,NaN,NaN,False,Berlin,Germany
4,5,Graphic Designer,None,7,True,No salary info,2023-04-22,2024-08-26,60000.0,80000.0,70000.0,False,London,UK


## 3. Limpar Tabelas

In [3]:
cur.execute("""
    TRUNCATE TABLE
      gold.fact_job_posting,
      gold.dim_company,
      gold.dim_job_title,
      gold.dim_location,
      gold.dim_fraud_reason
    CASCADE;
""")
conn.commit()

print("Tabelas GOLD limpas")


Tabelas GOLD limpas


In [4]:
companies = (
    df[["company_name"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_company (company_name) VALUES (%s)",
    [(c.strip(),) for c in companies["company_name"] if str(c).strip()],
    page_size=5000
)
conn.commit()


In [5]:
titles = (
    df[["job_title"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_job_title (job_title) VALUES (%s)",
    [(t.strip(),) for t in titles["job_title"] if str(t).strip()],
    page_size=5000
)
conn.commit()


In [6]:
locations = df[["country", "state"]].drop_duplicates()

rows = []
for _, r in locations.iterrows():
    if pd.notna(r["country"]) or pd.notna(r["state"]):
        rows.append((r["country"], r["state"]))

execute_batch(
    cur,
    "INSERT INTO gold.dim_location (country, state) VALUES (%s, %s)",
    rows,
    page_size=5000
)
conn.commit()


In [7]:
reasons = (
    df[["fraud_reason"]]
    .dropna()
    .drop_duplicates()
)

execute_batch(
    cur,
    "INSERT INTO gold.dim_fraud_reason (fraud_reason) VALUES (%s)",
    [(r.strip(),) for r in reasons["fraud_reason"] if str(r).strip()],
    page_size=5000
)
conn.commit()


In [8]:
cur.execute("SELECT company_name, company_sk FROM gold.dim_company")
company_map = dict(cur.fetchall())

cur.execute("SELECT job_title, job_title_sk FROM gold.dim_job_title")
title_map = dict(cur.fetchall())

cur.execute("SELECT country, state, location_sk FROM gold.dim_location")
location_map = {(c, s): k for c, s, k in cur.fetchall()}

cur.execute("SELECT fraud_reason, fraud_reason_sk FROM gold.dim_fraud_reason")
fraud_map = dict(cur.fetchall())


In [9]:
rows = []

for _, r in df.iterrows():
    rows.append((
        int(r.job_id),
        company_map.get(r.company_name),
        title_map.get(r.job_title),
        location_map.get((r.country, r.state)),
        fraud_map.get(r.fraud_reason),
        r.salary_avg,
        r.remote,
        r.is_fake
    ))

execute_batch(
    cur,
    """
    INSERT INTO gold.fact_job_posting (
        job_id,
        company_srk,
        job_title_srk,
        location_srk,
        fraud_reason_srk,
        salary_avg,
        remote,
        is_fake
    )
    VALUES (%s,%s,%s,%s,%s,%s,%s,%s)
    """,
    rows,
    page_size=5000
)

conn.commit()
print("FACT carregada com sucesso")


FACT carregada com sucesso


In [10]:
print("\nGOLD (apos ETL):")
for t in [
    "gold.dim_company",
    "gold.dim_job_title",
    "gold.dim_location",
    "gold.dim_fraud_reason",
    "gold.fact_job_posting",
]:
    cur.execute(f"SELECT COUNT(*) FROM {t}")
    print(f"  {t}: {cur.fetchone()[0]:,}")



GOLD (apos ETL):
  gold.dim_company: 880
  gold.dim_job_title: 10
  gold.dim_location: 4
  gold.dim_fraud_reason: 3
  gold.fact_job_posting: 3,000


In [11]:
cur.close()
conn.close()
print("\nETL finalizado com sucesso ✅")


ETL finalizado com sucesso ✅
